# Contour Tests

In [1]:
# Type imports
from typing import Any, Dict, List, Tuple

# Standard Libraries
from pathlib import Path
from collections import Counter, defaultdict
import math
from math import sqrt, pi, sin, cos, tan, radians
from statistics import mean
from itertools import zip_longest

# Shared Packages
import pandas as pd
import numpy as np
import xlwings as xw

import pygraphviz as pgv
import networkx as nx

In [2]:
from types_and_classes import *

## functions for creating contours

In [3]:
def circle_points(radius: float, offset_x: float = 0, offset_y: float = 0,
                  num_points: int = 16, precision=3)->list[tuple[float, float]]:
    deg_step = radians(360/num_points)
    degree_points = np.arange(stop=radians(360), step=deg_step)
    x_coord = np.array([round(radius*sin(d), precision) for d in degree_points])
    y_coord = np.array([round(radius*cos(d), precision) for d in degree_points])

    x_coord = x_coord + offset_x
    y_coord = y_coord + offset_y
    coords = [(x,y) for x,y in zip(x_coord,y_coord)]
    return coords

In [4]:
def box_points(width:float, height: float = None, offset_x: float = 0,
               offset_y: float = 0) -> list[tuple[float, float]]:
    x1_unit = width / 2
    if not height:
        y1_unit = x1_unit
    else:
        y1_unit = height / 2
    coords = [
        ( x1_unit + offset_x,  y1_unit + offset_y),
        ( x1_unit + offset_x, -y1_unit + offset_y),
        (-x1_unit + offset_x, -y1_unit + offset_y),
        (-x1_unit + offset_x,  y1_unit + offset_y)
        ]
    return coords

In [10]:
box1_def = {
        'struct_id': 'GTV',
        'roi': 38,
        'structure_type': 'GTV',
        'structure_code': 'GTVp',
        'structure_code_meaning': 'Primary Gross Tumor Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (255, 0, 0),
        'volume': 8.03,
        'length': 2.6,
        'sup_slice': -0.4,
        'inf_slice': -3,
        'center_of_mass': (-5.36,  9.71, -1.63)
        }

structure_def = Structure(**box1_def)


38

In [12]:
box6 = shapely.Polygon(box_points(6))
box4 = shapely.Polygon(box_points(4))
hollow_box = shapely.difference(box6, box4)
structure_2d: StructureSlice = shapely.MultiPolygon([hollow_box])

In [14]:
slices = [-1.0, -0.5, 0.5, 1.0]
data_list = []
for slice_idx in slices:
    data_item = {
        'ROI Num': structure_def.roi_num,
        'Slice Index': SliceIndex(slice_idx),
        'Structure Slice': structure_2d
        }
    data_list.append(data_item)
slice_data = pd.DataFrame(data_list)
slice_data.set_index(['ROI Num', 'Slice Index'], inplace=True)
slice_data

Structure Slice
ROI Num Slice Index                                                   
38      -1.0         MULTIPOLYGON (((3 3, 3 -3, -3 -3, -3 3, 3 3), ...
        -0.5         MULTIPOLYGON (((3 3, 3 -3, -3 -3, -3 3, 3 3), ...
         0.5         MULTIPOLYGON (((3 3, 3 -3, -3 -3, -3 3, 3 3), ...
         1.0         MULTIPOLYGON (((3 3, 3 -3, -3 -3, -3 3, 3 3), ...

ContourData: Table
	Index: AutoInteger
	Columns:
		ROI_Num,
		SliceIndex,
		Area,
		Contour
Generated by: Read Contour Data

StructureData: Series:
	Index: ROI_Num, SliceIndex
	Values: StructureSlice
Generated by: Build StructureSet


In [ ]:
structure_def = [
    {
        'struct_id': 'GTV',
        'roi': 38,
        'structure_type': 'GTV',
        'structure_code': 'GTVp',
        'structure_code_meaning': 'Primary Gross Tumor Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (255, 0, 0),
        'volume': 8.03,
        'length': 2.6,
        'sup_slice': -0.4,
        'inf_slice': -3,
        'center_of_mass': (-5.36,  9.71, -1.63)
        },{
        'struct_id': 'CTV',
        'roi': 24,
        'structure_type': 'GTV',
        'structure_code': 'ITV',
        'structure_code_meaning': 'Internal Target Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (255, 255, 0),
        'volume': 34.45,
        'length': 3.6,
        'sup_slice': 0,
        'inf_slice': -3.6,
        'center_of_mass': (-5.34,  9.79, -1.61)
        },{
        'struct_id': 'PTV',
        'roi': 30,
        'structure_type': 'PTV',
        'structure_code': 'PTVp',
        'structure_code_meaning': 'Primary Planning Target Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (0, 255, 255),
        'volume': 74.649,
        'length': 4.6,
        'sup_slice': 0.6,
        'inf_slice': -4.0,
        'center_of_mass': (-5.36,  9.78, -1.59)
        },{
        'struct_id': 'eval PTV',
        'roi': 41,
        'structure_type': 'PTV',
        'structure_code': 'PTVp',
        'structure_code_meaning': 'Primary Planning Target Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (0, 255, 255),
        'volume': 74.649,
        'length': 4.6,
        'sup_slice': 0.6,
        'inf_slice': -4.0,
        'center_of_mass': (-5.36,  9.78, -1.59)
        },{
        'struct_id': 'BODY',
        'roi': 1,
        'structure_type': 'EXTERNAL',
        'structure_code': 'BODY',
        'structure_code_meaning': 'Body',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (0, 255, 0),
        'volume': 28951.626,
        'length': 33.8,
        'sup_slice': 10.6,
        'inf_slice': -23.2,
        'center_of_mass': (-0.95,  9.73, -6.76)
        },{
        'struct_id': 'Lung L',
        'roi': 26,
        'structure_type': 'ORGAN',
        'structure_code': '7310',
        'structure_code_meaning': 'Left lung',
        'structure_code_scheme': 'FMA',
        'color': (224, 255, 255),
        'volume': 1776,
        'length': 24.2,
        'sup_slice': 5.8,
        'inf_slice': -18.4,
        'center_of_mass': (7.08, 10.61, -6.28)
        },{
        'struct_id': 'Lung R',
        'roi': 27,
        'structure_type': 'ORGAN',
        'structure_code': '7309',
        'structure_code_meaning': 'Right lung',
        'structure_code_scheme': 'FMA',
        'color': (255, 218, 185),
        'volume': 2556.676,
        'length': 23.8,
        'sup_slice': 6.6,
        'inf_slice': -17.2,
        'center_of_mass': (-8.09,  8.77, -5.57)
        },{
        'struct_id': 'Lung B',
        'roi': 25,
        'structure_type': 'ORGAN',
        'structure_code': '68877',
        'structure_code_meaning': 'Pair of lungs',
        'structure_code_scheme': 'FMA',
        'color': (218, 165, 32),
        'volume': 4332.676,
        'length': 25,
        'sup_slice': 6.6,
        'inf_slice': -18.4,
        'center_of_mass': (-1.87,  9.52, -5.86)
        },{
        'struct_id': 'Skin',
        'roi': 2,
        'structure_type': 'ORGAN',
        'structure_code': '7163',
        'structure_code_meaning': 'Skin',
        'structure_code_scheme': 'FMA',
        'color': (240, 255, 240),
        'volume': 1726.808,
        'length': 33.8,
        'sup_slice': 10.6,
        'inf_slice': -23.2,
        'center_of_mass': (-0.95,  9.73, -6.76),
        'show': False
        }
    ]

structure_list = []

for node_dict in structure_def:
    roi = node_dict['roi']
    struct_id = node_dict['struct_id']
    structure_ref = {'roi': roi, 'struct_id': struct_id,
                     'Structure': Structure(**node_dict)}
    structure_list.append(structure_ref)

structure_table = pd.DataFrame(structure_list)
structure_table.set_index('roi', inplace=True)

In [ ]:
edge_def = [
    {'structures': (1, 2),   'relationship': 'CONFINES',      'is_logical': False, 'show': True},
    {'structures': (1, 25),  'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (1, 27),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (1, 26),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (1, 30),  'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (1, 41),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (1, 24),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (1, 38),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (2, 25),  'relationship': 'DISJOINT',      'is_logical': False, 'show': True},
    {'structures': (2, 27),  'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (2, 26),  'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (2, 30),  'relationship': 'SURROUNDS',     'is_logical': False, 'show': False},
    {'structures': (2, 41),  'relationship': 'SURROUNDS',     'is_logical': True , 'show': True},
    {'structures': (2, 24),  'relationship': 'SURROUNDS',     'is_logical': True , 'show': False},
    {'structures': (2, 38),  'relationship': 'SURROUNDS',     'is_logical': True , 'show': False},
    {'structures': (25, 27), 'relationship': 'CONTAINS',      'is_logical': False, 'show': True},
    {'structures': (25, 26), 'relationship': 'CONTAINS',      'is_logical': False, 'show': True},
    {'structures': (25, 30), 'relationship': 'OVERLAPS',      'is_logical': True , 'show': False},
    {'structures': (25, 41), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (25, 24), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (25, 38), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (27, 26), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (27, 30), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (27, 41), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (27, 24), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (27, 38), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (26, 30), 'relationship': 'OVERLAPS',      'is_logical': False, 'show': True},
    {'structures': (26, 41), 'relationship': 'INCORPORATES',  'is_logical': True , 'show': True},
    {'structures': (26, 24), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (26, 38), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (30, 41), 'relationship': 'INCORPORATES',  'is_logical': False, 'show': True},
    {'structures': (30, 24), 'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (30, 38), 'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (41, 24), 'relationship': 'CONTAINS',      'is_logical': False, 'show': True},
    {'structures': (41, 38), 'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (24, 38), 'relationship': 'CONTAINS',      'is_logical': False, 'show': True},
    ]


relationship_list = []

for edge_dict in edge_def:
    relationship = Relationship(**edge_dict)
    relationship_type = edge_dict['relationship']
    edge = list(edge_dict['structures']) + [relationship_type, relationship]
    relationship_list.append(edge)
relationship_table = pd.DataFrame(relationship_list)
relationship_table.columns = ['ROI_1', 'ROI_2', 'Relationship Type',
                              'Relationship']
relationship_table.set_index(['ROI_1', 'ROI_2'], inplace=True)